In [2]:
!pip show pymongo

Name: pymongo
Version: 3.10.1
Summary: Python driver for MongoDB <http://www.mongodb.org>
Home-page: http://github.com/mongodb/mongo-python-driver
Author: Mike Dirolf
Author-email: mongodb-user@googlegroups.com
License: Apache License, Version 2.0
Location: c:\anaconda3\lib\site-packages
Requires: 
Required-by: 


You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


---
### Pymongo를 사용해서 MongoDB 제어하기
1. pymongo라이브러리 import
2. MongoDB 접속
3. database, Collection 생성
4. Collection의 document를 CRUD하기



---
### MongoDB 연결하고 db, collection 생성

In [10]:
import pymongo

# mongoDB 연결
conn = pymongo.MongoClient(host="localhost", port=27017)
print(conn) 

# database 생성
db = conn.pymongo_db 
print(db) 
print(db.name)

# collection 생성
col = db.pymongo_col
print(col)
print(col.name)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'pymongo_db')
pymongo_db
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'pymongo_db'), 'pymongo_col')
pymongo_col


---
### Insert_one(), insert_many()

In [22]:
post = {"author": "길동", "text":"첫번째 글", "tags":["mongodb", "python", "pymongo"]}
result = col.insert_one(post)
print(result) # 객체를 출력 ( type이 나옴)
print(result.inserted_id) # 넣은 document 의 아이디를 조회

5f22621d3c8e4047a3981b96


In [14]:
col.find()

In [15]:
for post in col.find():
    print(type(post))
    print(post)

<class 'dict'>
{'_id': ObjectId('5f225fd83c8e4047a3981b91'), 'author': '길동', 'text': '첫번째 글', 'tags': ['mongodb', 'python', 'pymongo']}


In [17]:
# document 갯수 알아내기
col.count_documents({})

1

In [19]:
# insert many 사용 : 여러개의 document를 한꺼번에 등록
posts = [
    {"author":"둘리", "age": 10},
    {"author":"바다", "age": 20},
    {"author":"연못", "age": 30},
]
col.insert_many(posts)

In [23]:
print(col.count_documents({}))

for post in col.find():
    print(post)
    

6
{'_id': ObjectId('5f225fd83c8e4047a3981b91'), 'author': '길동', 'text': '첫번째 글', 'tags': ['mongodb', 'python', 'pymongo']}
{'_id': ObjectId('5f2261653c8e4047a3981b92'), 'author': '둘리', 'age': 10}
{'_id': ObjectId('5f2261653c8e4047a3981b93'), 'author': '바다', 'age': 20}
{'_id': ObjectId('5f2261653c8e4047a3981b94'), 'author': '연못', 'age': 30}
{'_id': ObjectId('5f2261fb3c8e4047a3981b95'), 'author': '길동', 'text': '첫번째 글', 'tags': ['mongodb', 'python', 'pymongo']}
{'_id': ObjectId('5f22621d3c8e4047a3981b96'), 'author': '길동', 'text': '첫번째 글', 'tags': ['mongodb', 'python', 'pymongo']}


In [34]:
for post in col.find({},{"author":1, "age":1, "text":1, "_id":0})\
.sort("age", pymongo.ASCENDING):
    print(post)

{'author': '길동', 'text': '첫번째 글'}
{'author': '길동', 'text': '첫번째 글'}
{'author': '길동', 'text': '첫번째 글'}
{'author': '둘리', 'age': 10}
{'author': '바다', 'age': 20}
{'author': '연못', 'age': 30}


In [36]:
# employees collection 생성하기
emp = db.employees
print(emp)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'pymongo_db'), 'employees')


In [76]:
import datetime # 날짜 데이터 포맷을 위함

emp_list = [
{"number":1001,"last_name":"Smith","first_name":"John","salary":62000,"department":"sales", "hire_date":datetime.datetime.strptime("2016-01-02",'%Y-%m-%d')},
{"number":1002,"last_name":"Anderson","first_name":"Jane","salary":57500,"department":"marketing", "hire_date":datetime.datetime.strptime("2013-11-09",'%Y-%m-%d')},
{"number":1003,"last_name":"Everest","first_name":"Brad","salary":71000,"department":"sales", "hire_date":datetime.datetime.strptime("2017-02-03",'%Y-%m-%d')},
{"number":1004,"last_name":"Horvath","first_name":"Jack","salary":42000,"department":"marketing", "hire_date":datetime.datetime.strptime("2017-06-01",'%Y-%m-%d')},
]
emp.insert_many(emp_list)
# emp.delete_many({})

In [77]:
for e in emp.find({},{"_id":0}):
    print(e)

{'number': 1001, 'last_name': 'Smith', 'first_name': 'John', 'salary': 62000, 'department': 'sales', 'hire_date': datetime.datetime(2016, 1, 2, 0, 0)}
{'number': 1002, 'last_name': 'Anderson', 'first_name': 'Jane', 'salary': 57500, 'department': 'marketing', 'hire_date': datetime.datetime(2013, 11, 9, 0, 0)}
{'number': 1003, 'last_name': 'Everest', 'first_name': 'Brad', 'salary': 71000, 'department': 'sales', 'hire_date': datetime.datetime(2017, 2, 3, 0, 0)}
{'number': 1004, 'last_name': 'Horvath', 'first_name': 'Jack', 'salary': 42000, 'department': 'marketing', 'hire_date': datetime.datetime(2017, 6, 1, 0, 0)}


In [78]:
# first_name like %a%
filt = {"first_name" : {"$regex": "a"}}
filt = {"first_name" : {"$regex": "^J"}}
filt = {"last_name" : {"$regex": "h$"}}

from datetime import datetime
from_dt = datetime.strptime("2016-01-02","%Y-%m-%d")
to_dt = datetime.strptime("2017-05-30","%Y-%m-%d")

filt = {
        "hire_date":{"$gte": from_dt},
        "hire_date":{ "$lte": to_dt}
       }
filt = {"hire_date":{"$gte": from_dt, "$lte": to_dt}}

filt = {
    "number":{"$in":[1001,1002]}
}
for empolyee in emp.find(filt, {"_id":0}):
    print(empolyee)

{'number': 1001, 'last_name': 'Smith', 'first_name': 'John', 'salary': 62000, 'department': 'sales', 'hire_date': datetime.datetime(2016, 1, 2, 0, 0)}
{'number': 1002, 'last_name': 'Anderson', 'first_name': 'Jane', 'salary': 57500, 'department': 'marketing', 'hire_date': datetime.datetime(2013, 11, 9, 0, 0)}


---
### update_one(), update_many()

In [79]:
# 1001번의 부서를 바꾸기
emp.update_one({"number":1001}, {"$set":{"department":"marketing"}})

# number=1001 의 salary 62000 -> 63000
emp.update_one({"number":1001},{"$set":{"salary": 63000}})

In [80]:
emp.update_many({},{"$set":{"status":"A"}})

In [82]:
for employee in emp.find({},{"_id":0}):
    print(employee)

{'number': 1001, 'last_name': 'Smith', 'first_name': 'John', 'salary': 63000, 'department': 'marketing', 'hire_date': datetime.datetime(2016, 1, 2, 0, 0), 'status': 'A'}
{'number': 1002, 'last_name': 'Anderson', 'first_name': 'Jane', 'salary': 57500, 'department': 'marketing', 'hire_date': datetime.datetime(2013, 11, 9, 0, 0), 'status': 'A'}
{'number': 1003, 'last_name': 'Everest', 'first_name': 'Brad', 'salary': 71000, 'department': 'sales', 'hire_date': datetime.datetime(2017, 2, 3, 0, 0), 'status': 'A'}
{'number': 1004, 'last_name': 'Horvath', 'first_name': 'Jack', 'salary': 42000, 'department': 'marketing', 'hire_date': datetime.datetime(2017, 6, 1, 0, 0), 'status': 'A'}


In [83]:
# update_many()
# number in [1003, 1004] status = 'B' 로 update()
emp.update_many({"number":{"$in":[1003,1004]}},{"$set":{"status":'B'}})

In [84]:
for employee in emp.find({},{"_id":0}):
    print(employee)

{'number': 1001, 'last_name': 'Smith', 'first_name': 'John', 'salary': 63000, 'department': 'marketing', 'hire_date': datetime.datetime(2016, 1, 2, 0, 0), 'status': 'A'}
{'number': 1002, 'last_name': 'Anderson', 'first_name': 'Jane', 'salary': 57500, 'department': 'marketing', 'hire_date': datetime.datetime(2013, 11, 9, 0, 0), 'status': 'A'}
{'number': 1003, 'last_name': 'Everest', 'first_name': 'Brad', 'salary': 71000, 'department': 'sales', 'hire_date': datetime.datetime(2017, 2, 3, 0, 0), 'status': 'B'}
{'number': 1004, 'last_name': 'Horvath', 'first_name': 'Jack', 'salary': 42000, 'department': 'marketing', 'hire_date': datetime.datetime(2017, 6, 1, 0, 0), 'status': 'B'}


---
### delete_many()

In [85]:
# delete_many()
# status = 'B'
emp.delete_many({"status": 'B'})

In [86]:
for employee in emp.find({},{"_id":0}):
    print(employee)

{'number': 1001, 'last_name': 'Smith', 'first_name': 'John', 'salary': 63000, 'department': 'marketing', 'hire_date': datetime.datetime(2016, 1, 2, 0, 0), 'status': 'A'}
{'number': 1002, 'last_name': 'Anderson', 'first_name': 'Jane', 'salary': 57500, 'department': 'marketing', 'hire_date': datetime.datetime(2013, 11, 9, 0, 0), 'status': 'A'}
